<a href="https://colab.research.google.com/github/vaishakhRaveendran/Sus_hack/blob/main/cotton_india.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna --quiet
!pip install umap-learn --quiet

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,RandomForestClassifier
from xgboost import XGBRegressor
from sklearn.impute import KNNImputer,SimpleImputer
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import sklearn
sklearn.set_config(transform_output="pandas")
from sklearn.preprocessing import LabelEncoder
import optuna
from functools import partial
import gc

In [ ]:
FILEPATH='https://raw.githubusercontent.com/vaishakhRaveendran/Sus_hack/main'

In [ ]:
df_train = pd.read_csv(os.path.join(FILEPATH,'India_train.csv'))
df_test = pd.read_csv(os.path.join(FILEPATH, 'India_test.csv'))

In [ ]:
df_train.columns.tolist()

In [ ]:
from sklearn.pipeline import FunctionTransformer
#Advanced Pipelines
from sklearn import set_config
set_config(display="diagram")
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.compose import ColumnTransformer,make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder,MinMaxScaler

class adv_pipe():
  def __init__(self):
    return None

  def create_pipe(self):
    self.adv_ppl=Pipeline(steps=[
     ('Column_transform',ColumnTransformer(transformers=[
           ('num',make_pipeline(SimpleImputer(strategy='constant'),MinMaxScaler()),make_column_selector(dtype_include='float')),
           ('cat',make_pipeline(SimpleImputer(strategy='most_frequent'),OrdinalEncoder()),make_column_selector(dtype_include='object'))],
    remainder='passthrough',
    verbose_feature_names_out=False)),
    ('Reduce_col',reduce_col(0.007)),
    ('dimension_reduction',Decomp())
    ])

    return self.adv_ppl



In [ ]:
class Groups():
  def __init__(self,df_train,df_test,category,random_state=42,test_size=0.33):
    self.category=category
    self.gp_train=df_train.groupby(self.category)
    self.gp_test=df_test.groupby(self.category)
    self.random_state=random_state
    self.test_size=test_size
    self.target_list=df_train[self.category].unique().tolist()

  def Split(self,impute=False):
    if impute:
       for name in self.target_list:
        A=self.gp_train.get_group(name)
        B=self.gp_test.get_group(name)
        yield A,B

    else:
      for state in self.target_list:
        ppl=adv_pipe().create_pipe()
        X=self.gp_train.get_group(state).drop(['COTTON YIELD (Kg per ha)'],inplace=False,axis=1)
        y=self.gp_train.get_group(state)['COTTON YIELD (Kg per ha)']
        test= self.gp_test.get_group(state)
        X=ppl.fit_transform(X)
        test=ppl.transform(test)
        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, random_state=42)
        yield state,X_train, X_test, y_train, y_test,test


In [ ]:
#Normal transformers cant be used for stateful transformations
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.feature_selection import VarianceThreshold
class reduce_col(TransformerMixin):
  def __init__(self,threshold=0.0):
    self.threshold=threshold

  def fit(self,X,y=None):
     col_vars=X.var()
     self.col_to_drop = col_vars[col_vars.values<self.threshold].index
     return self

  def transform(self,X):
    assert self.col_to_drop is not None, 'Drop_col error, must be fitted before predict'
    X.drop(self.col_to_drop, axis=1, inplace=True)
    return X



In [ ]:
import umap.umap_ as umap
class Decomp(BaseEstimator, TransformerMixin):
    def __init__(self, n_components=20, method="umap", col_drop=['Year','COTTON PRODUCTION (1000 tons)']):
        self.n_components = n_components
        self.method = method
        self.col_drop = col_drop

    def fit(self, X, y=None):
        X_temp = X.drop(self.col_drop, axis=1) if self.col_drop else X.copy()
        if self.method == "pca":
            self.comp = PCA(n_components=self.n_components, random_state=0)
        elif self.method == "umap":
            self.comp = umap.UMAP(n_components=self.n_components, random_state=0)
        else:
            raise ValueError(f"Invalid method name: {self.method}")
        self.comp.fit(X_temp)
        return self

    def transform(self, X):
        X_temp = pd.concat([X.pop(x) for x in self.col_drop], axis=1)
        X_array = np.asarray(X)
        X_reduced = self.comp.transform(X_array)
        columns = [f"{self.method.upper()}_{i}" for i in range(self.n_components)]
        X_reduced = pd.DataFrame(X_reduced, columns=columns, index=X_temp.index)
        return pd.concat([X_temp,X_reduced],axis=1)


In [ ]:
class Regressor:
    def __init__(self):
        self.models = self._define_model()
        self.models_name = list(self._define_model().keys())
        self.len_models = len(self.models)

    def _define_model(self):

        xgb_params = {
        }
        lgbm_params={

        }


        rf_params =  {


        }

        gdbt_params = {

        }


        models = {
            'xgb':  XGBRegressor(**xgb_params),
            'gdbt': GradientBoostingRegressor(**gdbt_params),
            'rf' : RandomForestRegressor(**rf_params),
            'lgbm': LGBMRegressor(**lgbm_params)
        }

        return models

In [ ]:
class OptunaWeights:
    def __init__(self, random_state, n_trials=2000):
        self.study = None
        self.weights = None
        self.random_state = random_state
        self.n_trials = n_trials

    def _objective(self, trial, y_true, y_preds):
        # Define the weights for the predictions from each model
        weights = [trial.suggest_float(f"weight{n}", 1e-14, 1) for n in range(len(y_preds))]

        # Calculate the weighted prediction
        weighted_pred = np.average(np.array(y_preds).T, axis=1, weights=weights)

        # Calculate the score for the weighted prediction
        score = mean_absolute_error(y_true, weighted_pred)
        return score

    def fit(self, y_true, y_preds):
        optuna.logging.set_verbosity(optuna.logging.ERROR)
        sampler = optuna.samplers.CmaEsSampler(seed=self.random_state)
        pruner = optuna.pruners.HyperbandPruner()
        self.study = optuna.create_study(sampler=sampler, pruner=pruner, study_name="OptunaWeights", direction='minimize')
        objective_partial = partial(self._objective, y_true=y_true, y_preds=y_preds)
        self.study.optimize(objective_partial, n_trials=self.n_trials)
        self.weights = [self.study.best_params[f"weight{n}"] for n in range(len(y_preds))]

    def predict(self, y_preds):
        assert self.weights is not None, 'OptunaWeights error, must be fitted before predict'
        weighted_pred = np.average(np.array(y_preds).T, axis=1, weights=self.weights)
        return weighted_pred

    def fit_predict(self, y_true, y_preds):
        self.fit(y_true, y_preds)
        return self.predict(y_preds)

    def weights(self):
        return self.weights

In [ ]:
Gp=Groups(df_train,df_test,'State Name',10,0.4)
test_predss={}
ensemble_score={}
for i, (state,X_train, X_val, y_train, y_val,test) in enumerate(Gp.Split()):
    regressor = Regressor()
    models = regressor.models
    test_preds = []
    oof_preds = []
    print(state,X_train.columns)
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_val_pred =model.predict(X_val).reshape(-1)
        test_pred = model.predict(test).reshape(-1)
        oof_preds.append(y_val_pred)
        test_preds.append(test_pred)

    optweights = OptunaWeights(random_state=42)
    y_val_pred = optweights.fit_predict(y_val.values, oof_preds)
    score = mean_squared_error(y_val, y_val_pred)
    ensemble_score[state]=score
    test_predss[state]=optweights.predict(test_preds)
    gc.collect()


In [ ]:
ensemble_score

In [ ]:
np.array(list(ensemble_score.values())).mean()